In [1]:
import pandas as pd


You are a data analyst in **Stripe**'s risk management team investigating transaction patterns to identify potential fraud. The team needs to develop a systematic approach to screen transactions for financial risks. Your goal is to create an initial risk assessment methodology using transaction characteristics.

In [2]:
# Load the datasets
dim_risk_flags = pd.read_csv('dim_risk_flags.csv')
fct_transactions = pd.read_csv('fct_transactions.csv')

# Display the datasets
print("Dimension: Risk Flags")
print(dim_risk_flags)
print("\nFact: Transactions")
print(fct_transactions)


Dimension: Risk Flags
  risk_level  risk_flag_id  transaction_id
0        Low             1               2
1     Medium             2               7
2       High             3              11
3       High             4              12
4       High             5              13
5     Medium             6              14
6       High             7              15
7        Low             8               1
8     Medium             9               6
9        Low            10               3

Fact: Transactions
          customer_email  transaction_id transaction_date  transaction_amount  \
0        alice@gmail.com               1       2024-10-05              120.00   
1   bob@customdomain.com               2       2024-10-15              250.50   
2      charlie@yahoo.com               3       2024-10-20               75.25   
3       dana@hotmail.com               4       2024-10-25              100.00   
4            eve@biz.org               5       2024-10-30              300.00   

### Question 1 of 3

How many transactions in October 2024 have a customer email ending with a domain other than 'gmail.com', 'yahoo.com', or 'hotmail.com'? This metric will help us identify transactions associated with less common email providers that may indicate emerging risk patterns.

In [3]:
# Q1: Count transactions in Oct 2024 with non-common email domains — other than ".gmail.com"., ".yahoo.com"., or ".hotmail.com"

# Ensure transaction_date is in datetime format
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

# Extract domains from customer_email
allowed_domains = {'gmail.com', 'yahoo.com', 'hotmail.com'}
domains = (
    fct_transactions['customer_email']
    .astype(str)
    .str.extract(r'@([^@]+)$', expand=False)
    .str.lower()
)

# Filter for October 2024 transactions
mask_oct_2024 = (
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 10)
)

# Create a mask for other email domains
mask_other_domains = ~domains.isin(allowed_domains)

# Combine masks to count transactions
count_other = (mask_oct_2024 & mask_other_domains).sum()

# Display the result
print("Transactions in Oct 2024 with non-common email domains:", int(count_other))


Transactions in Oct 2024 with non-common email domains: 2


### Question 2 of 3

For transactions occurring in November 2024, what is the average transaction amount, using 0 as a default for any missing values? This calculation will help us detect abnormal transaction amounts that could be related to fraudulent activity.

In [4]:
# Q2: Average transaction amount in Nov 2024, treating missing amounts as 0

# Ensure transaction_date is in datetime format
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

# Ensure transaction_amount is numeric
amounts = pd.to_numeric(fct_transactions['transaction_amount'], errors='coerce')

# Create a mask for November 2024 transactions
mask_nov_2024 = (
    (fct_transactions['transaction_date'].dt.year == 2024) &
    (fct_transactions['transaction_date'].dt.month == 11)
)

# Create a series for November 2024 transaction amounts
series_nov = amounts.loc[mask_nov_2024].fillna(0)

# Calculate the average transaction amount
avg_amount_nov = series_nov.mean() if len(series_nov) else 0.0

# Display the result
print(f"Average transaction amount in Nov 2024 (missing as 0): {avg_amount_nov:.2f}")


Average transaction amount in Nov 2024 (missing as 0): 180.15


### Question 3 of 3

Among transactions flagged as 'High' risk in December 2024, which day of the week recorded the highest number of such transactions? This analysis is intended to pinpoint specific days with concentrated high-risk activity and support the development of our preliminary fraud detection score.

In [5]:
# Q3: Day of week with most 'High' risk transactions in Dec 2024

# Ensure transaction_date is in datetime format
fct_transactions['transaction_date'] = pd.to_datetime(fct_transactions['transaction_date'], errors='coerce')

# Join the transactions with risk flags
df_join = fct_transactions.merge(
    dim_risk_flags[['transaction_id', 'risk_level']],
    on='transaction_id',
    how='inner'
)

# Deduplicate by transaction to avoid counting multiple flags per transaction
df_high = df_join[df_join['risk_level'].str.lower() == 'high'].drop_duplicates(subset=['transaction_id'])

# Create a mask for December 2024 transactions
mask_dec_2024 = (
    (df_high['transaction_date'].dt.year == 2024) &
    (df_high['transaction_date'].dt.month == 12)
)

# Count occurrences by day of the week
dow_counts = (
    df_high.loc[mask_dec_2024, 'transaction_date']
    .dt.day_name()
    .value_counts()
)

# Identify the day with the most 'High' risk transactions
if not dow_counts.empty:
    top_day = dow_counts.idxmax()
    top_count = int(dow_counts.max())
    print(f"Day with most 'High' risk transactions in Dec 2024: {top_day} ({top_count})")
else:
    print("No 'High' risk transactions found in Dec 2024.")

# Optional: distribution for reference
print("Counts by day:", dow_counts.to_dict())


Day with most 'High' risk transactions in Dec 2024: Monday (3)
Counts by day: {'Monday': 3, 'Tuesday': 1}
